In [ ]:
!pip install pdfminer3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pymysql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
 !pip install streamlit -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.0.17 requires typing-extensions<4.2.0,>=3.7.4.1; python_version < "3.8", but you have typing-extensions 4.4.0 which is incompatible.
spacy 3.0.8 requires typing-extensions<4.0.0.0,>=3.7.4; python_version < "3.8", but you have typing-extensions 4.4.0 which is incompatible.
confection 0.0.3 requires typing-extensions<4.2.0,>=3.7.4.1; python_version < "3.8", but you have typing-extensions 4.4.0 which is incompatible.


In [ ]:
!pip install pyresparser

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached typing_extensions-3.10.0.2-py3-none-any.whl (26 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.4.0
    Uninstalling typing-extensions-4.4.0:
      Successfully uninstalled typing-extensions-4.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
rich 12.6.0 requires typing-extensions<5.0,>=4.0.0; python_version < "3.9", but you have typing-extensions 3.10.0.2 which is incompatible.


In [ ]:
!pip install spacy==3.1.1


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached spacy-3.1.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.4 MB)
  Attempting uninstall: spacy
    Found existing installation: spacy 3.0.8
    Uninstalling spacy-3.0.8:
      Successfully uninstalled spacy-3.0.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.0.0 requires spacy<3.1.0,>=3.0.0, but you have spacy 3.1.1 which is incompatible.


In [ ]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.0.0/en_core_web_sm-3.0.0.tar.gz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.7 MB 4.9 MB/s 
  Using cached spacy-3.0.8-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.8 MB)
  Attempting uninstall: spacy
    Found existing installation: spacy 3.1.1
    Uninstalling spacy-3.1.1:
      Successfully uninstalled spacy-3.1.1


In [ ]:
!pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%%writefile app.py

from pdfminer3.pdfinterp import PDFResourceManager
from pdfminer3.pdfinterp import PDFPageInterpreter
from pdfminer3.converter import TextConverter
from pdfminer3.pdfpage import PDFPage
from pdfminer3.layout import LAParams, LTTextBox
import io,random
import base64,random
import pymysql
import streamlit as st
import nltk
nltk.download('stopwords')
from pyresparser import ResumeParser
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
# from scipy.sparse import hstack



def pdf_reader(file):
    resource_manager = PDFResourceManager()
    fake_file_handle = io.StringIO()
    converter = TextConverter(resource_manager, fake_file_handle, laparams=LAParams())
    page_interpreter = PDFPageInterpreter(resource_manager, converter)
    with open(file, 'rb') as fh:
        for page in PDFPage.get_pages(fh,
                                      caching=True,
                                      check_extractable=True):
            page_interpreter.process_page(page)
            print(page)
        text = fake_file_handle.getvalue()

    # close open handles
    converter.close()
    fake_file_handle.close()
    return text


def show_pdf(file_path):
    with open(file_path, "rb") as f:
        base64_pdf = base64.b64encode(f.read()).decode('utf-8')
    # pdf_display = f'<embed src="data:application/pdf;base64,{base64_pdf}" width="700" height="1000" type="application/pdf">'
    pdf_display = F'<iframe src="data:application/pdf;base64,{base64_pdf}" width="700" height="1000" type="application/pdf"></iframe>'
    st.markdown(pdf_display, unsafe_allow_html=True)



## Loading data  

data=pd.read_csv('UpdatedResumeDataSet.csv')

categoryList = []

for i in data['Category'].unique():
    categoryList.append(i)







## Plotting the Pie chart

count=data['Category'].value_counts()
label=data['Category'].value_counts().keys()

plt.figure(1, figsize=(25,25))
grid=GridSpec(2,2)

cmap=plt.get_cmap('coolwarm')

color=[cmap(i) for i in np.linspace(0, 1, 5)]
plt.subplot(grid[0,1], aspect=1, title='Distribution')

pie=plt.pie(count, labels=label, autopct='%1.2f%%')
#plt.show()






## Data cleaning

import re

def clean(text):
    text=re.sub('http\S+\s*', ' ', text)
    text=re.sub('RT|cc', ' ', text)
    text=re.sub('#\S+', '', text)
    text=re.sub('@\S+', '', text)
    text=re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', text)
    text=re.sub('\s+', ' ', text)
    text=re.sub(r'[^\x00-\x7f]', r' ', text)
    return text

data['clean text']=data.Resume.apply(lambda x: clean(x))






## Categories labelled as numbers

#print(print(data['Category'].unique()))
var=['Category']
le=LabelEncoder()

for i in var:
    data[i]=le.fit_transform(data[i])


#print(print(data['Category'].unique()))  


categoryDict = {6: "Data scientist",12:"HR",0:"Advocate",1:"Arts",24:"Web Designing",16:"Mechanical Engineer",22:"Sales",14:"Health and fitness",5:"Civil Engineer",

                    15:"Java Developer",4:"Business Analyst",21:"SAP Developer",2:"Automation Testing",11:"Electrical Engineering",18:"Operations Manager",
                    
                  
                    20:"Python Developer",8:"DevOps Engineer",17:"Network Security Engineer",19:"PMO",7:"Database",13:"Hadoop",10:"ETL Developer",9:"DotNet Developer",3:"Blockchain",23:"Testing"}





# Vectorization

text=data['clean text'].values
print(text.shape)
terget=data['Category'].values

vect=TfidfVectorizer(
    sublinear_tf=True,
    stop_words='english',
    max_features=2000)

Word_feature=vect.fit_transform(text)




# Splitting data

x_train, x_test, y_train, y_test=train_test_split(Word_feature, terget, random_state=0, test_size=0.2)
print(x_train.shape)
print(x_test.shape)




# model_selection

import sklearn
from sklearn.multiclass import OneVsOneClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier

model=OneVsRestClassifier(KNeighborsClassifier())
model.fit(x_train, y_train)
prediction=model.predict(x_test)



test_accuracy = metrics.accuracy_score(y_test, prediction)

test_accuracy = "{:.2f}".format(test_accuracy*100);


st.header("Welcome to CV Analysis System")

st.write("This system has a test accuracy of "+str(test_accuracy)+" %")

CategoryNum = str(len(categoryList))
st.header("***We have a total of ***"+CategoryNum +"*** job positions including:***")
#for i in range(len(categoryList)):
    #st.write(i+1," "+categoryList[i])
st.set_option('deprecation.showPyplotGlobalUse', False)
st.pyplot(plt.show())




pdf_file = st.file_uploader("Choose your Resume", type=["pdf"])

if pdf_file is not None:
            # with st.spinner('Uploading your Resume....'):
            #     time.sleep(4)
            save_image_path = './content'+pdf_file.name
            with open(save_image_path, "wb") as f:
                f.write(pdf_file.getbuffer())
            show_pdf(save_image_path)
                ## Get the whole resume data
            resume_text = pdf_reader(save_image_path)

            #cv_text = clean(resume_text)
            text = np.array([resume_text])

            Word_feature=vect.transform(text)
            prediction=model.predict(Word_feature)
            #category = categoryDict[prediction]
            st.header("**Resume Analysis**")
            job = categoryDict[prediction[0]]
            st.write(job)
            st.header("Out system predict from your CV that you are suitable for the post as a "+job)
            
            st.subheader("**Your Basic info**")
            st.write(resume_text)
            


Overwriting app.py


In [ ]:
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pyngrok import ngrok 

ngrok.set_auth_token("2GLPdUs6oCiTljktFgbOlMBKxbA_6TWW3ishdvvXymwTaiobQ")

ngrok.kill()

In [ ]:
!nohup streamlit run app.py --server.port 80 & 
url = ngrok.connect(port = 80)

print(url)

nohup: appending output to 'nohup.out'
NgrokTunnel: "http://e3e8-34-67-65-144.ngrok.io" -> "http://localhost:80"


In [ ]:
from pyngrok import ngrok 

tunnels = ngrok.get_tunnels()

tunnels

[<NgrokTunnel: "https://9393-34-67-65-144.ngrok.io" -> "http://localhost:80">,
 <NgrokTunnel: "http://9393-34-67-65-144.ngrok.io" -> "http://localhost:80">]